# Test SQL statements

In [1]:
%load_ext sql

In [2]:
import boto3
import configparser
import matplotlib.pyplot as plt
import pandas as pd
import json
import time
import psycopg2
from sql_queries import *

## Assume cluster is already created.
## Create connection to DB

In [3]:
# Read config files
config = configparser.ConfigParser()
config.read('dwh.cfg')
host, dbname, user, = config.get("DWH","dwh_endpoint"), config.get("DWH", "dwh_db"),  config.get("DWH", "dwh_db_user")
password, port =  config.get("DWH", "dwh_db_password"),  config.get("DWH", "dwh_port")
(host, dbname, user, password, port)

arn_role = config.get("DWH", "dwh_role_arn")

In [5]:
# Connect to cloud
connection_string = "host={} dbname={} user={} password={} port={}".format(host, dbname, user, password, port)
conn = psycopg2.connect(connection_string)
cur = conn.cursor()
conn.set_session(autocommit=True)

print("Connection Established.")

Connection Established.


## Close connection

In [26]:
conn.close()

## Create Schemas and Tables

In [6]:
# Step 1. Create schemas
sql_query = staging_schema_create
cur.execute(sql_query)
conn.commit()

sql_query = star_schema_create
cur.execute(sql_query)
conn.commit()

# Step 2. Create all tables
for statement in create_table_queries:
    cur.execute(statement)
    conn.commit()

## DROP Tables

In [17]:
for statement in drop_table_queries:
    cur.execute(statement)
    conn.commit()

## Import data from S3 bucket to staging tables

In [7]:
for query in copy_table_queries:
    print(query)
    cur.execute(query)
    conn.commit()


COPY staging_schema.staging_events
FROM 's3://udacity-dend/log_data'
credentials 'aws_iam_role=arn:aws:iam::904972942839:role/dwhRole'
FORMAT AS json 's3://udacity-dend/log_json_path.json'
region 'us-west-2';


COPY staging_schema.staging_songs
FROM 's3://udacity-dend/song_data'
credentials 'aws_iam_role=arn:aws:iam::904972942839:role/dwhRole'
FORMAT AS json 'auto'
region 'us-west-2';



# Transform the data and load to star schema tables

## Use the list from sql_queries

In [9]:
for query in insert_table_queries:
    print(query)
    cur.execute(query)
    conn.commit()


-- Time code taken from: https://stackoverflow.com/questions/39815425/how-to-convert-epoch-to-datetime-redshift
INSERT INTO star_schema.songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
SELECT DISTINCT
       TIMESTAMP 'epoch' + (se.ts / 1000) * INTERVAL '1 second' as start_time,
                se.userId,
                se.level,
                ss.song_id,
                ss.artist_id,
                se.sessionId,
                se.location,
                se.userAgent
FROM staging_schema.staging_songs ss
INNER JOIN staging_schema.staging_events se
ON (ss.title = se.song AND se.artist = ss.artist_name)
AND se.page = 'NextSong';


INSERT INTO star_schema.users
SELECT DISTINCT userId, firstName, lastName, gender, level
FROM staging_schema.staging_events
WHERE userId IS NOT NULL
AND page = 'NextSong';


INSERT INTO star_schema.songs
SELECT DISTINCT song_id, title, artist_id, year, duration
FROM staging_schema.staging_songs
WHERE song_id IS

### Load to time table

In [8]:
sql_query = """
-- Import data from staging_events -> time
-- Conversion from ms to TIMESTAMP taken from:
-- https://stackoverflow.com/questions/39815425/how-to-convert-epoch-to-datetime-redshift
INSERT INTO star_schema.time
SELECT DISTINCT
       TIMESTAMP 'epoch' + (ts/1000) * INTERVAL '1 second' as start_time,
       EXTRACT(HOUR FROM start_time) AS hour,
       EXTRACT(DAY FROM start_time) AS day,
       EXTRACT(WEEKS FROM start_time) AS week,
       EXTRACT(MONTH FROM start_time) AS month,
       EXTRACT(YEAR FROM start_time) AS year,
       to_char(start_time, 'Day') AS weekday
FROM staging_schema.staging_events
"""

print(sql_query)

cur.execute(sql_query)
conn.commit()


-- Import data from staging_events -> time
-- Conversion from ms to TIMESTAMP taken from:
-- https://stackoverflow.com/questions/39815425/how-to-convert-epoch-to-datetime-redshift
INSERT INTO star_schema.time
SELECT DISTINCT
       TIMESTAMP 'epoch' + (ts/1000) * INTERVAL '1 second' as start_time,
       EXTRACT(HOUR FROM start_time) AS hour,
       EXTRACT(DAY FROM start_time) AS day,
       EXTRACT(WEEKS FROM start_time) AS week,
       EXTRACT(MONTH FROM start_time) AS month,
       EXTRACT(YEAR FROM start_time) AS year,
       to_char(start_time, 'Day') AS weekday
FROM staging_schema.staging_events



## Load to users table

In [10]:
sql_query = """
INSERT INTO star_schema.users
SELECT DISTINCT userId, firstName, lastName, gender, level
FROM staging_schema.staging_events
WHERE userId IS NOT NULL
AND page = 'NextSong';
"""

print(sql_query)

cur.execute(sql_query)
conn.commit()


INSERT INTO star_schema.users
SELECT DISTINCT userId, firstName, lastName, gender, level
FROM staging_schema.staging_events
WHERE userId IS NOT NULL
AND page = 'NextSong';



## Load to song table

In [12]:
sql_query = """
INSERT INTO star_schema.songs
SELECT DISTINCT song_id, title, artist_id, year, duration
FROM staging_schema.staging_songs
WHERE song_id IS NOT NULL;
"""

print(sql_query)

cur.execute(sql_query)
conn.commit()


INSERT INTO star_schema.songs
SELECT DISTINCT song_id, title, artist_id, year, duration
FROM staging_schema.staging_songs
WHERE song_id IS NOT NULL;



## Load into artists table

In [13]:
sql_query = """
INSERT INTO star_schema.artists
SELECT DISTINCT artist_id, artist_name, artist_location, artist_latitude, artist_longitude
FROM staging_schema.staging_songs;
"""

print(sql_query)

cur.execute(sql_query)
conn.commit()


INSERT INTO star_schema.artists
SELECT DISTINCT artist_id, artist_name, artist_location, artist_latitude, artist_longitude
FROM staging_schema.staging_songs;



## Load into songplays table

In [16]:
sql_query = """
-- Time code taken from: https://stackoverflow.com/questions/39815425/how-to-convert-epoch-to-datetime-redshift
INSERT INTO star_schema.songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
SELECT DISTINCT
       TIMESTAMP 'epoch' + (se.ts / 1000) * INTERVAL '1 second' as start_time,
                se.userId,
                se.level,
                ss.song_id,
                ss.artist_id,
                se.sessionId,
                se.location,
                se.userAgent
FROM staging_schema.staging_songs ss
INNER JOIN staging_schema.staging_events se
ON (ss.title = se.song AND se.artist = ss.artist_name)
AND se.page = 'NextSong';
"""

print(sql_query)

cur.execute(sql_query)
conn.commit()


-- Time code taken from: https://stackoverflow.com/questions/39815425/how-to-convert-epoch-to-datetime-redshift
INSERT INTO star_schema.songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
SELECT DISTINCT
       TIMESTAMP 'epoch' + (se.ts / 1000) * INTERVAL '1 second' as start_time,
                se.userId,
                se.level,
                ss.song_id,
                ss.artist_id,
                se.sessionId,
                se.location,
                se.userAgent
FROM staging_schema.staging_songs ss
INNER JOIN staging_schema.staging_events se
ON (ss.title = se.song AND se.artist = ss.artist_name)
AND se.page = 'NextSong';

